In [1]:
import datetime
time = datetime.datetime.now()
print("launched at : ", time)
import pandas as pd
import numpy as np

#!pip3 install surprise
from surprise import *
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split

def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x, mean_x, std_x

print("IMPORT DONE")

# n_jobs = int(input("Value for how many processors to use : (-1 is all, -2 is all except one) "))
# print("")

############
df = pd.read_csv("Datasets/data_train.csv")

df[["user", "item"]] = df.Id.str.split("_", expand=True)

df.user = df.user.str.replace("r", "")
df.item = df.item.str.replace("c", "")

reader = Reader(rating_scale=(1,5)) 
df_7, df_3 = train_test_split(df, train_size=0.7, random_state=1)

tmp7 = Dataset.load_from_df(df_7[["user","item","Prediction"]], reader)
tmp3 = Dataset.load_from_df(df_3[["user","item","Prediction"]], reader)
data_train_7 = tmp7.build_full_trainset()
data_train_3 = tmp3.build_full_trainset()
print("DATA AND READER ARE READY")
print("")


launched at :  2019-12-17 18:45:42.278807
IMPORT DONE
DATA AND READER ARE READY



In [2]:
df2 = pd.read_csv("Datasets/sample_submission.csv")

df2[["user", "item"]] = df2.Id.str.split("_", expand=True)

df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")

reader = Reader(rating_scale=(1,5)) 
data_test = Dataset.load_from_df(df2[["user","item","Prediction"]], reader)
test = data_test.construct_testset(data_test.raw_ratings)

In [3]:
X = np.load("npy/X_2.npy")
pred_array = np.load("npy/pred_array_2.npy")

In [4]:
y=df_3.Prediction.values
clf=RidgeCV(alphas=np.linspace(10**-5,1,10),cv=10)
clf=clf.fit(X,y)

In [5]:
clf.coef_

array([-0.21966809,  0.17497274, -0.04549438,  0.27422431,  0.64131247,
        0.27095068,  0.28502718, -0.29926261])

In [6]:
pred=clf.predict(pred_array.T)
final_array=np.rint(pred)

In [7]:
final_array[np.where(final_array>5)]=5
final_array[np.where(final_array<1)]=1

In [8]:
final_array[final_array<3]

array([2., 2., 2., ..., 2., 2., 2.])

In [9]:
np.unique(final_array,return_counts=True)


(array([1., 2., 3., 4., 5.]), array([   131,  13638, 268764, 773732, 120687]))

In [54]:
df2.Prediction = final_array
df2 = df2.drop(columns=["user", "item"])
df2.to_csv("Datasets/submission_run_script.csv", index=False)

print("EVERYTHING DONE")

EVERYTHING DONE
